# Extracting coarse-grained classifiers from large CNN models.

The following example shows how to obtain corarse-grained classes with original models trained on ImageNet. This serves as a baseline in our experiment. In this case, we do not extract new models, but use solely the original models, and then map the prediction based on the WrodNet hierarchy (After making a prediction).

Import of all the necessary libraries:

In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

from nltk.corpus import wordnet as wn
import nltk
from tensorflow.keras.applications.imagenet_utils import decode_predictions
from keras.utils import to_categorical

Below, we include the implementation of the function that allows to find all the hyponyms (sub-classes) of a given WordNet node. We use the nltk WordNet implementation.

In [2]:
def find_hyponyms(hypernym_name):
    """
    Function can be used to find all the hyponyms of a given WordNet node, 
    which is a hypernym of some classes.
    Parameters
    ----------
    hypernym_name - name of a WordNet node for which we aim to find all available 
    hyponyms in ImageNet
    
    Returns
    -------
    ids_array - a NumPy array with ImageNet ids of desired fine-grained classes (hyponyms)
    """
    hyponyms = wn.synsets(hypernym_name)[0]
    hyponyms = set([i for i in hyponyms.closure(lambda s:s.hyponyms())])
    offsets = []
    imagenet_classes = decode_predictions(to_categorical(np.expand_dims(np.array(range(1000)), axis=-1), num_classes=1000), top=1)

    for c in imagenet_classes:
        offsets.append(int(c[0][0].split('n')[1]))
    
    ids = []
    for idx, o in enumerate(offsets):
        isadoggo = wn.synset_from_pos_and_offset('n', int(o))
        if isadoggo in hyponyms:
            ids.append(idx)
    ids_array = np.array(ids)
    return ids_array

Read an example model from Keras (MobileNetV2)

In [3]:
model = MobileNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling='avg',
    classes=1000,
    classifier_activation="softmax"
)

Read a dataset (we use image_dataset_from_directory from tf.keras.preprocessing to create a generator). In the example below, we use our own very small dataset, but to gather the results for the purpose of our paper, we used Kaggle Dogs vs. Cats dataset (see https://www.kaggle.com/c/dogs-vs-cats) - it has the same format as our tiny example.

In [8]:
# set a correct image size for a network (MobileNetV2)
image_size = (224, 224)
batch_size = 32

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    "PetImages",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
    label_mode='categorical',
    shuffle=False
)
normalized_ds = test_ds.map(lambda x, y: (preprocess_input(x), y))

# here, we read the labels for testing (correct only when shuffle is False)
labels = np.concatenate([y for x, y in normalized_ds], axis=0) 

Found 20 files belonging to 2 classes.


Below, we generate predictions with the original model:

In [5]:
predictions = model.predict(normalized_ds)
y_predicted = np.argmax(predictions, axis=1)

1/1 [==============================] - 1s 1s/step


Below, we provide an example implementation that maps the ImageNet classes to classes Dog, Cat and Other (all classes besides dogs and cats).

In [6]:
def map_classes(prediction):
    dog_classes = find_hyponyms(hypernym_name='dog')
    cat_classes = find_hyponyms(hypernym_name='cat')
    d = {}
    
    for i in range(1000):
        if i in cat_classes:
            d[i] = 0
        elif i in dog_classes:
            d[i] = 1
        else:
            d[i] = 2
            
    mapped = np.vectorize(d.get)(prediction)
    return mapped

In this case, our confusion matrix is 3 x 3 - we also include class 'other'. In the case of cats (index 0), the network mistakes them for dogs and other ImageNet classes.

In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix
true_y =  np.argmax(labels, axis=1)
predicted_y = map_classes(y_predicted)
print('Accuracy')
print(accuracy_score(true_y, predicted_y))
print('Confusion matrix')
print(confusion_matrix(true_y, predicted_y, normalize='true'))

Accuracy
0.75
Confusion matrix
[[0.5 0.3 0.2]
 [0.  1.  0. ]
 [0.  0.  0. ]]
